In [ ]:
!pip install faker

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from faker import Faker
from tqdm.notebook import tqdm

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data = {
    'ticket_id':[],
    'date':[],
    'customer_id':[],
    'gender':[],
    'dob':[],
    'age':[],
    'branch_id':[],
    'province':[],
    'purity':[],
    'weight':[],
    'no_of_article':[],
    'loan_amount':[],
    'interest_rate':[],
    'interest_accrued':[],
    # 'interest_paid':[],
    # 'interest_balance':[],
    'no_of_installement':[],
    'installement_amount':[],
    'monthly_paid_amount':[],
    'outstanding_amount':[],
    'status':[],
    'pound_rate':[],
    'dollar_rate':[]
}

In [ ]:
fake = Faker()

In [ ]:
def generate_random_date(start_date, end_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(end_date, "%Y-%m-%d")

    # Calculate the difference in days between start and end dates
    days_difference = (end_datetime - start_datetime).days

    # Generate a random number of days within the range
    random_days = random.randint(0, days_difference)

    # Calculate the random date
    random_date = start_datetime + timedelta(days=random_days)

    return random_date.strftime("%Y-%m-%d")

    # #Example: Generate a random date between 2022-01-01 and 2023-01-01
    # start_date = "2021-04-01"
    # end_date = "2023-03-31"


In [ ]:
import yfinance as yf

def get_high_values(start_date):
    end_date = '2023-11-30'
    symbol = 'LKR=X'

    try:
        exchange_rate_data = yf.download(symbol, start=start_date, end=end_date)
        high_values = exchange_rate_data['High'][0]
    except Exception as e:
        print(f"Error: {e}")
        high_values = 0

    return high_values

In [ ]:
def get_gold_values(start_date, exchange_rate_data):
    end_date = '2023-11-30'
    gold_ticker = 'GC=F'

    try:
        gold_rate_data = yf.download(gold_ticker, start=start_date, end=end_date)
        gold_values = gold_rate_data['High'][0]
        result = gold_values * exchange_rate_data * 8 / 28.3495
    except Exception as e:
        print(f"Error: {e}")
        result = 0

    return result

In [ ]:
no_records = 14500
start_date_1 = "2020-01-01"
end_date_1 = "2022-10-31"

start_ticket_no = 5001

for ticket_no in tqdm(range(start_ticket_no, start_ticket_no + no_records), desc="Processing tickets"):
    print("ITERATION OUTER LOOP: ", ticket_no)
    customer_id = np.random.randint(2001,15000)
    branch_id = np.random.randint(1,500)
    province = np.random.choice(['Central Province', 'Eastern Province', 'Northern Province', 'North Central Province', 'North Province', 'Western Province', 'Sabaragamuwa Province', 'Southern Province', 'Uva Province','Central Province', 'Western Province'])
    purity = np.random.choice(['22','24'])
    weight = np.random.randint(2,40)
    no_of_article = np.random.randint(1,10)
    interest_rate_value = np.random.uniform(0.15,0.3)
    no_of_installement = np.random.choice([3,6,12])
    gender = fake.random_element(['Male', 'Female'])
    birthdate = fake.date_of_birth(minimum_age=23, maximum_age=75)
    birthdate_datetime = datetime.combine(birthdate, datetime.min.time())
    age = (datetime.now() - birthdate_datetime).days // 365
    dob = birthdate.strftime('%Y-%m-%d')

    #data['date'] = []
    start_date = generate_random_date(start_date_1, end_date_1)
    start_date = pd.to_datetime(start_date)

    start_date_i=start_date
    dollar_rate_initial = get_high_values(start_date_i)
    pound_rate_initial = get_gold_values(start_date_i,dollar_rate_initial)
    loan_amount_value = (weight*pound_rate_initial*0.8)/8


    interest_accrued= loan_amount_value * interest_rate_value
    total_amount_to_pay = loan_amount_value + interest_accrued
    installement_amount = total_amount_to_pay / no_of_installement
    no_months =  no_of_installement

    for month in range(1, no_months + 1):
      print("ITERATION INNER LOOP:", month)

      data['ticket_id'].append(ticket_no)
      data['customer_id'].append(customer_id)
      data['branch_id'].append(branch_id)
      data['province'].append(province)
      data['purity'].append(purity)
      data['weight'].append(weight)
      data['no_of_article'].append(no_of_article)
      data['loan_amount'].append(loan_amount_value)
      data['interest_rate'].append(interest_rate_value)
      data['no_of_installement'].append(no_of_installement)
      data['gender'].append(gender)
      data['dob'].append(dob)
      data['age'].append(age)
      data['interest_accrued'].append(interest_accrued)
      data['installement_amount'].append(installement_amount)



      date_value = start_date + pd.DateOffset(months=1)
      data['date'].append(date_value)
      start_date = date_value

      # Loan-related calculations

      #Define a range for random adjustments (you can adjust this based on your requirements)
      random_adjustment_range = 0.1  # 10% variation, for example

      # Generate a random value between -random_adjustment_range and random_adjustment_range
      random_adjustment = random.uniform(-random_adjustment_range, random_adjustment_range)

      # Apply the random adjustment to each element of the installment amount list
      monthly_paid_amount = installement_amount+ (installement_amount) * random_adjustment
      data['monthly_paid_amount'].append(monthly_paid_amount)


      #Outstanding Amount

      outstanding_amount = total_amount_to_pay - monthly_paid_amount
      total_amount_to_pay = outstanding_amount

      if (outstanding_amount > 0) and  (no_months > month) :
        status='Active'    #data['status'] = 'Active'   #status = 'Active'

      elif (outstanding_amount > 0) and (no_months == month):
        status='Auction'

      elif (outstanding_amount < 0):
        outstanding_amount = 0
        status ='Redeemed'

      data['status'].append(status)
      data['outstanding_amount'].append(outstanding_amount)

      dollar_rate = get_high_values(date_value)
      data['dollar_rate'].append(dollar_rate)

      pound_rate = get_gold_values(date_value,dollar_rate)
      data['pound_rate'].append(pound_rate)




Processing tickets:   0%|          | 0/2000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[*********************100%%**********************]  1 of 1 completed
ITERATION INNER LOOP: 4
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ITERATION INNER LOOP: 5
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ITERATION INNER LOOP: 6
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ITERATION OUTER LOOP:  25304
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ITERATION INNER LOOP: 1
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ITERATION INNER LOOP: 2
[*********************100%%**************

In [ ]:
 for key, value in data.items():
    print(f'Length of {key}: {len(value)}')

Length of ticket_id: 14022
Length of date: 14022
Length of customer_id: 14022
Length of gender: 14022
Length of dob: 14022
Length of age: 14022
Length of branch_id: 14022
Length of province: 14022
Length of purity: 14022
Length of weight: 14022
Length of no_of_article: 14022
Length of loan_amount: 14022
Length of interest_rate: 14022
Length of interest_accrued: 14022
Length of no_of_installement: 14022
Length of installement_amount: 14022
Length of monthly_paid_amount: 14022
Length of outstanding_amount: 14022
Length of status: 14022
Length of pound_rate: 14022
Length of dollar_rate: 14022


In [ ]:
data

{'ticket_id': [23500,
  23500,
  23500,
  23501,
  23501,
  23501,
  23502,
  23502,
  23502,
  23503,
  23503,
  23503,
  23504,
  23504,
  23504,
  23504,
  23504,
  23504,
  23505,
  23505,
  23505,
  23505,
  23505,
  23505,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23506,
  23507,
  23507,
  23507,
  23508,
  23508,
  23508,
  23509,
  23509,
  23509,
  23509,
  23509,
  23509,
  23510,
  23510,
  23510,
  23510,
  23510,
  23510,
  23511,
  23511,
  23511,
  23512,
  23512,
  23512,
  23512,
  23512,
  23512,
  23513,
  23513,
  23513,
  23514,
  23514,
  23514,
  23514,
  23514,
  23514,
  23515,
  23515,
  23515,
  23516,
  23516,
  23516,
  23516,
  23516,
  23516,
  23517,
  23517,
  23517,
  23517,
  23517,
  23517,
  23518,
  23518,
  23518,
  23518,
  23518,
  23518,
  23519,
  23519,
  23519,
  23519,
  23519,
  23519,
  23520,
  23520,
  23520,
  23520,
  23520,
  23520,
  23521,
  23521,
  23521,
  23521,
  2352

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head(70
        )

,ticket_id,date,customer_id,gender,dob,age,branch_id,province,purity,weight,no_of_article,loan_amount,interest_rate,interest_accrued,no_of_installement,installement_amount,monthly_paid_amount,outstanding_amount,status,pound_rate,dollar_rate
0,23500,2021-03-14,28390,Female,1980-09-05,43,71,Southern Province,24,34,7,335397.120754,0.185399,62182.227064,3,132526.449273,143978.099183,253601.248635,Active,95042.869860,194.469788
1,23500,2021-04-14,28390,Female,1980-09-05,43,71,Southern Province,24,34,7,335397.120754,0.185399,62182.227064,3,132526.449273,145561.399723,108039.848912,Active,97833.904561,198.575394
2,23500,2021-05-14,28390,Female,1980-09-05,43,71,Southern Province,24,34,7,335397.120754,0.185399,62182.227064,3,132526.449273,131865.099178,0.000000,Redeemed,100934.684268,194.085938
3,23501,2022-05-01,28202,Male,1964-06-16,59,235,Sabaragamuwa Province,24,34,2,547570.080064,0.221834,121469.648788,3,223013.242950,232694.766107,436344.962744,Active,187937.689634,351.113678
4,23501,2022-06-01,28202,Male,1964-06-16,59,235,Sabaragamuwa Province,24,34,2,547570.080064,0.221834,121469.648788,3,223013.242950,211802.383268,224542.579476,Active,186765.073361,358.156311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,23513,2020-05-02,28045,Female,1948-04-27,75,291,Eastern Province,22,19,2,152306.099343,0.161374,24578.200740,3,58961.433361,60446.365488,0.000000,Redeemed,89812.873658,186.230988
66,23514,2021-06-30,28327,Male,1970-11-29,53,222,Uva Province,22,10,6,105661.829757,0.218895,23128.843891,6,21465.112275,21902.677377,106887.996270,Active,98258.645657,196.633118
67,23514,2021-07-30,28327,Male,1970-11-29,53,222,Uva Province,22,10,6,105661.829757,0.218895,23128.843891,6,21465.112275,22226.833782,84661.162488,Active,101526.239853,196.545898
68,23514,2021-08-30,28327,Male,1970-11-29,53,222,Uva Province,22,10,6,105661.829757,0.218895,23128.843891,6,21465.112275,19604.067261,65057.095227,Active,101077.127574,196.772919


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14022 entries, 0 to 14021
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ticket_id            14022 non-null  int64         
 1   date                 14022 non-null  datetime64[ns]
 2   customer_id          14022 non-null  int64         
 3   gender               14022 non-null  object        
 4   dob                  14022 non-null  object        
 5   age                  14022 non-null  int64         
 6   branch_id            14022 non-null  int64         
 7   province             14022 non-null  object        
 8   purity               14022 non-null  object        
 9   weight               14022 non-null  int64         
 10  no_of_article        14022 non-null  int64         
 11  loan_amount          14022 non-null  float64       
 12  interest_rate        14022 non-null  float64       
 13  interest_accrued     14022 non-

In [ ]:
df.to_csv('Gold_loan.csv', index=False)